In [16]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import json
from sklearn.linear_model import RidgeCV, Ridge, LinearRegression
from pathlib import Path
from scipy.stats import pearsonr
import pandas as pd
import nibabel as nib
from nilearn import plotting, datasets
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import random
import optuna 
import pickle
import gc

from train_utils import load_fmri, load_stimulus_features, align_features_and_fmri_samples, plot_correlations, train_model, training
from optimization import objective

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(777)

In [3]:
root_data_dir = 'data'
excluded_samples_start = 5  
excluded_samples_end = 5  
hrf_delay = 3
stimulus_window = 1
movies_train = ["friends-s01", "friends-s02", "friends-s03", "friends-s04", "friends-s05"] 
movies_test = ["friends-s06"] 

In [4]:
fmri_1 = load_fmri(root_data_dir, 1)
fmri_2 = load_fmri(root_data_dir, 2)
fmri_3 = load_fmri(root_data_dir, 3)
fmri_5 = load_fmri(root_data_dir, 5)

In [5]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_1')

5it [00:16,  3.22s/it]


(114294, 1536)
(23387, 1536)
8070144


In [6]:
x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


In [7]:
for k in y_train_1.keys():
    print(f"Train ROI {k}: {y_train_1[k].shape}, Test ROI {k}: {y_test_1[k].shape}")

Train ROI LH_IFGorb: (83980, 6), Test ROI LH_IFGorb: (17866, 6)
Train ROI LH_IFG: (83980, 11), Test ROI LH_IFG: (17866, 11)
Train ROI LH_MFG: (83980, 7), Test ROI LH_MFG: (17866, 7)
Train ROI LH_AntTemp: (83980, 17), Test ROI LH_AntTemp: (17866, 17)
Train ROI LH_PostTemp: (83980, 40), Test ROI LH_PostTemp: (17866, 40)


In [19]:
class OneHiddenMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
           # nn.BatchNorm1d(hidden_dim),
           # nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)
    


def mlp(input_dim, hidden_dim, output_dim, dropout_rate):
    return OneHiddenMLP(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        dropout_rate=dropout_rate
    )

In [23]:
def train_and_save_best_models(x_train_list, y_train_list, x_test_list, y_test_list,
                                study, layer, fedorenko_zone, save_dir='results',
                                num_epochs=20):
    
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Extract best hyperparameters directly from study
    lr = study.best_params['lr']
    decay = study.best_params['decay']
    hidden_dim = study.best_params['hidden_dim']
    batch_size = study.best_params['batch_size']
    dropout_rate = study.best_params['dropout_rate']
    
    
    subjects = [1, 2, 3, 5]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    models = []
    predictions_list = []
    
    for i, subject_id in enumerate(subjects):
        print(f"Training Subject {subject_id}")
        
        features_train = x_train_list[i]
        fmri_train = y_train_list[i]
        features_val = x_test_list[i]
        fmri_val = y_test_list[i]
        
        # Create model with best hyperparameters
        input_dim = features_train.shape[1]
        output_dim = fmri_train.shape[1]
        
        model = mlp(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout_rate=dropout_rate
        )
        
        # Train model
        trained_model = train_model(
            X_train=features_train,
            Y_train=fmri_train,
            X_val=features_val,
            Y_val=fmri_val,
            model_class=lambda: mlp(input_dim, hidden_dim, output_dim, dropout_rate),
            decay=decay,
            lr=lr,
            batch_size=batch_size,
            num_epochs=num_epochs,
            plot_curve=False
        )
        
        # Save model 
        model_save_path = save_dir / f'subject_{subject_id}_layer{layer}_zone{fedorenko_zone}_model.pth'
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'hyperparameters': {
                'input_dim': input_dim,
                'hidden_dim': hidden_dim,
                'output_dim': output_dim,
                'dropout_rate': dropout_rate,
                'lr': lr,
                'decay': decay,
                'batch_size': batch_size
            },
            'subject_id': subject_id
        }, model_save_path)
        print(f"Model saved: {model_save_path}")
        
        # Save complete model (for easier loading)
        model_full_path = save_dir / f'subject_{subject_id}_layer{layer}_zone{fedorenko_zone}_model_full.pkl'
        with open(model_full_path, 'wb') as f:
            pickle.dump(trained_model, f)
        print(f"Full model saved: {model_full_path}")
        
        # Generate predictions
        trained_model = trained_model.to(device)
        trained_model.eval()
        
        with torch.no_grad():
            X_test = torch.tensor(features_val, dtype=torch.float32).to(device)
            y_pred = trained_model(X_test)
            y_pred = y_pred.cpu().numpy()
        
        predictions_list.append(y_pred)
        
        # Keep model on CPU for Integrated Gradients
        trained_model = trained_model.cpu()
        models.append(trained_model)
        
        print(f"Subject {subject_id} - Predictions shape: {y_pred.shape}")
        
        # Cleanup
        del X_test
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
    
    return predictions_list

# Layer 1

## LH_IFGorb

In [9]:
subjects = [1, 2, 3, 5]
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

In [12]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_l1_IFGorb',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-11-25 13:40:28,104] A new study created in RDB with name: fmri_optimization_l1_IFGorb


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=1.70e-04, decay=7.31e-03, hidden=128, batch=1024

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [00:18<00:00,  1.08it/s]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:19<00:00,  1.03it/s]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:19<00:00,  1.04it/s]


 Final score (mean r): 0.0591
[I 2025-11-25 15:39:49,710] Trial 49 finished with value: 0.05912302806973457 and parameters: {'lr': 0.00018713372305030672, 'decay': 0.0008901122118262511, 'hidden_dim': 64, 'batch_size': 512, 'dropout_rate': 0.540928558590384}. Best is trial 7 with value: 0.06925341486930847.


In [15]:
study.best_params

{'lr': 0.00012244665354453448,
 'decay': 0.008792664501689107,
 'hidden_dim': 64,
 'batch_size': 512,
 'dropout_rate': 0.40719363653227614}

In [24]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_IFGorb', save_dir='results/layer1')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:19<00:00,  1.02it/s]


Model saved: results\layer1\subject_1_layer1_zoneLH_IFGorb_model.pth
Full model saved: results\layer1\subject_1_layer1_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:20<00:00,  1.02s/it]


Model saved: results\layer1\subject_2_layer1_zoneLH_IFGorb_model.pth
Full model saved: results\layer1\subject_2_layer1_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:51<00:00,  2.60s/it]


Model saved: results\layer1\subject_3_layer1_zoneLH_IFGorb_model.pth
Full model saved: results\layer1\subject_3_layer1_zoneLH_IFGorb_model_full.pkl
Subject 3 - Predictions shape: (17506, 6)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:20<00:00,  1.02s/it]


Model saved: results\layer1\subject_5_layer1_zoneLH_IFGorb_model.pth
Full model saved: results\layer1\subject_5_layer1_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)


In [27]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer1'
)

Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0765, mean MSE = 0.3422
  Subject 2: mean r = 0.0731, mean MSE = 0.3061
  Subject 3: mean r = 0.0729, mean MSE = 0.3628
  Subject 4: mean r = 0.0430, mean MSE = 0.4012
Mean across subjects: r = 0.0664, MSE = 0.3531
Saved: results\layer1\voxel_correlations_LH_IFGorb_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer1\correlation_map_LH_IFGorb_layer1.png


## LH_IFG

In [28]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

In [29]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_l1_IFG',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-11-25 16:48:09,654] A new study created in RDB with name: fmri_optimization_l1_IFG


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=1.41e-04, decay=4.76e-04, hidden=128, batch=2048

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [00:20<00:00,  1.00s/it]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:26<00:00,  1.30s/it]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:26<00:00,  1.31s/it]


 Final score (mean r): 0.1119
[I 2025-11-25 19:08:00,637] Trial 49 finished with value: 0.11186737567186356 and parameters: {'lr': 3.728895545005537e-05, 'decay': 0.009730656791798785, 'hidden_dim': 1024, 'batch_size': 2048, 'dropout_rate': 0.41561623246502055}. Best is trial 46 with value: 0.11592131108045578.


In [30]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_IFG', save_dir='results/layer1')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:21<00:00,  1.06s/it]


Model saved: results\layer1\subject_1_layer1_zoneLH_IFG_model.pth
Full model saved: results\layer1\subject_1_layer1_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:21<00:00,  1.05s/it]


Model saved: results\layer1\subject_2_layer1_zoneLH_IFG_model.pth
Full model saved: results\layer1\subject_2_layer1_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:21<00:00,  1.06s/it]


Model saved: results\layer1\subject_3_layer1_zoneLH_IFG_model.pth
Full model saved: results\layer1\subject_3_layer1_zoneLH_IFG_model_full.pkl
Subject 3 - Predictions shape: (17506, 11)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:21<00:00,  1.09s/it]


Model saved: results\layer1\subject_5_layer1_zoneLH_IFG_model.pth
Full model saved: results\layer1\subject_5_layer1_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)


In [32]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer1'
)

Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1136, mean MSE = 0.3362
  Subject 2: mean r = 0.1233, mean MSE = 0.3031
  Subject 3: mean r = 0.1259, mean MSE = 0.3909
  Subject 4: mean r = 0.0969, mean MSE = 0.3453
Mean across subjects: r = 0.1149, MSE = 0.3438
Saved: results\layer1\voxel_correlations_LH_IFG_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer1\correlation_map_LH_IFG_layer1.png


In [33]:
results

,schaefer_1000_id,correlation_sub1,p_value_sub1,significant_sub1,mse_sub1,correlation_sub2,p_value_sub2,significant_sub2,mse_sub2,correlation_sub3,...,significant_sub3,mse_sub3,correlation_sub4,p_value_sub4,significant_sub4,mse_sub4,correlation_mean,correlation_std,mse_mean,mse_std
0,266,0.100133,4.813084e-41,True,0.413600,0.034038,5.348244e-06,True,0.324078,0.072266,...,True,0.408750,0.033420,7.896731e-06,True,0.441898,0.059964,0.028025,0.397082,0.044009
1,341,0.087943,5.140010e-32,True,0.352841,0.120673,6.156982e-59,True,0.366754,0.103843,...,True,0.379573,0.072583,2.633273e-22,True,0.360236,0.096261,0.017912,0.364851,0.009822
2,344,0.213907,5.776934e-184,True,0.452031,0.190970,2.398518e-146,True,0.313761,0.239059,...,True,0.509147,0.136914,1.706012e-75,True,0.401114,0.195213,0.037712,0.419013,0.071786
3,345,0.108375,8.100653e-48,True,0.235571,0.148018,4.684122e-88,True,0.209491,0.122919,...,True,0.338502,0.114349,4.565933e-53,True,0.242349,0.123415,0.015116,0.256478,0.048919
4,347,0.106137,6.334564e-46,True,0.396961,0.127378,1.635309e-65,True,0.338700,0.163117,...,True,0.383844,0.155935,1.239393e-97,True,0.351373,0.138142,0.022806,0.367719,0.023582
5,348,0.102919,2.837572e-43,True,0.341327,0.145218,8.580504e-85,True,0.336961,0.149001,...,True,0.400043,0.087966,4.953490e-32,True,0.288289,0.121276,0.026403,0.341655,0.039621
6,351,0.087368,1.281527e-31,True,0.369358,0.123816,5.661546e-62,True,0.289782,0.151821,...,True,0.460947,0.116503,4.968699e-55,True,0.310272,0.119877,0.022936,0.357590,0.066442
7,353,0.092317,4.041831e-35,True,0.219064,0.111816,8.309386e-51,True,0.281495,0.114376,...,True,0.330294,0.106476,3.290567e-46,True,0.264515,0.106247,0.008532,0.273842,0.039790
8,432,0.180101,4.067274e-130,True,0.391956,0.128852,5.245266e-67,True,0.355209,0.097998,...,True,0.361111,0.102079,1.353536e-42,True,0.412988,0.127258,0.032730,0.380316,0.023463
9,433,0.097333,7.256355e-39,True,0.273502,0.130518,1.019925e-68,True,0.261310,0.090491,...,True,0.371161,0.069000,2.628985e-20,True,0.378816,0.096835,0.022079,0.321197,0.054032


## LH_MFG

In [34]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

In [ ]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_l1_MFG',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-11-25 19:59:50,096] A new study created in RDB with name: fmri_optimization_l1_MFG


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=8.86e-04, decay=5.22e-04, hidden=512, batch=2048

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [00:42<00:00,  2.12s/it]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.80s/it]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.90s/it]


 Final score (mean r): 0.0805
[I 2025-11-25 21:41:28,847] Trial 49 finished with value: 0.08046995103359222 and parameters: {'lr': 0.00047280851888542965, 'decay': 0.00854660973948551, 'hidden_dim': 64, 'batch_size': 512, 'dropout_rate': 0.5574870673714607}. Best is trial 13 with value: 0.0900309830904007.


In [36]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_MFG', save_dir='results/layer1')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:45<00:00,  2.25s/it]


Model saved: results\layer1\subject_1_layer1_zoneLH_MFG_model.pth
Full model saved: results\layer1\subject_1_layer1_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [01:05<00:00,  3.28s/it]


Model saved: results\layer1\subject_2_layer1_zoneLH_MFG_model.pth
Full model saved: results\layer1\subject_2_layer1_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.71s/it]


Model saved: results\layer1\subject_3_layer1_zoneLH_MFG_model.pth
Full model saved: results\layer1\subject_3_layer1_zoneLH_MFG_model_full.pkl
Subject 3 - Predictions shape: (17506, 7)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.90s/it]


Model saved: results\layer1\subject_5_layer1_zoneLH_MFG_model.pth
Full model saved: results\layer1\subject_5_layer1_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)


In [37]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer1'
)

Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.0982, mean MSE = 0.3122
  Subject 2: mean r = 0.0901, mean MSE = 0.2656
  Subject 3: mean r = 0.1030, mean MSE = 0.3209
  Subject 4: mean r = 0.0667, mean MSE = 0.3449
Mean across subjects: r = 0.0895, MSE = 0.3109
Saved: results\layer1\voxel_correlations_LH_MFG_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer1\correlation_map_LH_MFG_layer1.png


In [38]:
results

,schaefer_1000_id,correlation_sub1,p_value_sub1,significant_sub1,mse_sub1,correlation_sub2,p_value_sub2,significant_sub2,mse_sub2,correlation_sub3,...,significant_sub3,mse_sub3,correlation_sub4,p_value_sub4,significant_sub4,mse_sub4,correlation_mean,correlation_std,mse_mean,mse_std
0,124,0.033135,9.427071e-06,True,0.346850,0.099328,2.065241e-40,True,0.299704,0.056694,...,True,0.371200,0.021834,3.516659e-03,True,0.432162,0.052748,0.029688,0.362479,0.047741
1,130,0.047609,1.930122e-10,True,0.293294,0.042271,1.583445e-08,True,0.274471,0.056061,...,True,0.283432,0.017922,1.659769e-02,True,0.507966,0.040966,0.014184,0.339791,0.097324
2,134,0.033820,6.141374e-06,True,0.271392,0.040173,7.810541e-08,True,0.257120,0.046335,...,True,0.304873,0.010717,1.520424e-01,False,0.279371,0.032761,0.013475,0.278189,0.017346
3,223,0.114733,2.052060e-53,True,0.270857,0.115224,7.357440e-54,True,0.223493,0.129384,...,True,0.278008,0.103093,2.052040e-43,True,0.324398,0.115608,0.009318,0.274189,0.035766
4,225,0.101021,9.523064e-42,True,0.316149,0.136460,5.296888e-75,True,0.278171,0.105074,...,True,0.316505,0.101423,4.550703e-42,True,0.278441,0.110994,0.014787,0.297316,0.019012
5,233,0.211878,1.890452e-180,True,0.363515,0.090562,7.415921e-34,True,0.293864,0.137607,...,True,0.342047,0.062571,5.709823e-17,True,0.308775,0.125655,0.056543,0.327050,0.027340
6,457,0.144931,1.838750e-84,True,0.323417,0.106467,3.351407e-46,True,0.232176,0.189738,...,True,0.349956,0.149606,6.189645e-90,True,0.283073,0.147685,0.029490,0.297155,0.044436


## LH_AntTemp

In [39]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

In [40]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_l1_AntTemp',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-11-25 21:48:58,295] A new study created in RDB with name: fmri_optimization_l1_AntTemp


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=1.41e-04, decay=2.54e-04, hidden=256, batch=512

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.64s/it]


 Final score (mean r): 0.1858
[I 2025-11-25 23:33:09,364] Trial 49 finished with value: 0.18584932386875153 and parameters: {'lr': 0.00014219421694811177, 'decay': 0.008034730761196613, 'hidden_dim': 256, 'batch_size': 1024, 'dropout_rate': 0.4130733385979201}. Best is trial 43 with value: 0.1911395788192749.


In [ ]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_AntTemp', save_dir='results/layer1')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.75s/it]


Model saved: results\layer1\subject_1_layer1_zoneLH_AntTemp_model.pth
Full model saved: results\layer1\subject_1_layer1_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.89s/it]


Model saved: results\layer1\subject_2_layer1_zoneLH_AntTemp_model.pth
Full model saved: results\layer1\subject_2_layer1_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:33<00:00,  1.65s/it]


Model saved: results\layer1\subject_3_layer1_zoneLH_AntTemp_model.pth
Full model saved: results\layer1\subject_3_layer1_zoneLH_AntTemp_model_full.pkl
Subject 3 - Predictions shape: (17506, 17)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:33<00:00,  1.66s/it]


Model saved: results\layer1\subject_5_layer1_zoneLH_AntTemp_model.pth
Full model saved: results\layer1\subject_5_layer1_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)


In [42]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer1'
)

Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.1792, mean MSE = 0.2940
  Subject 2: mean r = 0.1791, mean MSE = 0.2926
  Subject 3: mean r = 0.2243, mean MSE = 0.3477
  Subject 4: mean r = 0.1804, mean MSE = 0.2974
Mean across subjects: r = 0.1908, MSE = 0.3079
Saved: results\layer1\voxel_correlations_LH_AntTemp_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer1\correlation_map_LH_AntTemp_layer1.png


## LH_PostTemp

In [43]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

In [44]:
study = optuna.create_study(
        direction='maximize',  
        study_name='fmri_optimization_l1_PostTemp',
        storage='sqlite:///optuna_mlp.db',  
        load_if_exists=True
    )

study.optimize(
        lambda trial: objective(x_train_list, y_train_list, x_test_list, y_test_list, trial), 
        n_trials=50,  
        timeout=None,  
        show_progress_bar=True
    )

[I 2025-11-25 23:42:07,479] A new study created in RDB with name: fmri_optimization_l1_PostTemp


  0%|          | 0/50 [00:00<?, ?it/s]


Trial 0
  lr=2.05e-05, decay=1.90e-04, hidden=512, batch=1024

=== Training Subject 1 ===


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.63s/it]



=== Training Subject 2 ===


...


=== Training Subject 3 ===


Training Epochs: 100%|██████████| 20/20 [00:33<00:00,  1.69s/it]



=== Training Subject 5 ===


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.70s/it]


 Final score (mean r): 0.1601
[I 2025-11-26 02:20:37,342] Trial 49 finished with value: 0.16011960804462433 and parameters: {'lr': 2.606985228194716e-05, 'decay': 0.0034757666833523707, 'hidden_dim': 512, 'batch_size': 2048, 'dropout_rate': 0.5914706696178637}. Best is trial 41 with value: 0.19102007150650024.


In [45]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=1, fedorenko_zone='LH_PostTemp', save_dir='results/layer1')

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:21<00:00,  1.06s/it]


Model saved: results\layer1\subject_1_layer1_zoneLH_PostTemp_model.pth
Full model saved: results\layer1\subject_1_layer1_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:20<00:00,  1.01s/it]


Model saved: results\layer1\subject_2_layer1_zoneLH_PostTemp_model.pth
Full model saved: results\layer1\subject_2_layer1_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


Training Epochs: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]


Model saved: results\layer1\subject_3_layer1_zoneLH_PostTemp_model.pth
Full model saved: results\layer1\subject_3_layer1_zoneLH_PostTemp_model_full.pkl
Subject 3 - Predictions shape: (17506, 40)
Training Subject 5


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.59s/it]


Model saved: results\layer1\subject_5_layer1_zoneLH_PostTemp_model.pth
Full model saved: results\layer1\subject_5_layer1_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)


In [46]:
results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=1,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer1'
)

Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1735, mean MSE = 0.3781
  Subject 2: mean r = 0.1856, mean MSE = 0.3504
  Subject 3: mean r = 0.2134, mean MSE = 0.4022
  Subject 4: mean r = 0.1883, mean MSE = 0.3631
Mean across subjects: r = 0.1902, MSE = 0.3734
Saved: results\layer1\voxel_correlations_LH_PostTemp_layer1_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer1\correlation_map_LH_PostTemp_layer1.png


# Layer 2

In [49]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_2')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:39,  7.94s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [52]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [53]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=2, fedorenko_zone='LH_IFGorb', save_dir='results/layer2')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=2,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer2'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:35<00:00,  1.77s/it]


Model saved: results\layer2\subject_1_layer2_zoneLH_IFGorb_model.pth
Full model saved: results\layer2\subject_1_layer2_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.82s/it]


Model saved: results\layer2\subject_2_layer2_zoneLH_IFGorb_model.pth
Full model saved: results\layer2\subject_2_layer2_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it]


Model saved: results\layer2\subject_5_layer2_zoneLH_IFGorb_model.pth
Full model saved: results\layer2\subject_5_layer2_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0806, mean MSE = 0.3419
  Subject 2: mean r = 0.0836, mean MSE = 0.3054
  Subject 3: mean r = 0.0792, mean MSE = 0.3626
  Subject 4: mean r = 0.0478, mean MSE = 0.4009
Mean across subjects: r = 0.0728, MSE = 0.3527
Saved: results\layer2\voxel_correlations_LH_IFGorb_layer2_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer2\correlation_map_LH_IFGorb_layer2.png


## LH_IFG

In [54]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [56]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=2, fedorenko_zone='LH_IFG', save_dir='results/layer2')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=2,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer2'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.60s/it]


Model saved: results\layer2\subject_1_layer2_zoneLH_IFG_model.pth
Full model saved: results\layer2\subject_1_layer2_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


Model saved: results\layer2\subject_2_layer2_zoneLH_IFG_model.pth
Full model saved: results\layer2\subject_2_layer2_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it]


Model saved: results\layer2\subject_5_layer2_zoneLH_IFG_model.pth
Full model saved: results\layer2\subject_5_layer2_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1216, mean MSE = 0.3353
  Subject 2: mean r = 0.1369, mean MSE = 0.3018
  Subject 3: mean r = 0.1351, mean MSE = 0.3897
  Subject 4: mean r = 0.1043, mean MSE = 0.3447
Mean across subjects: r = 0.1245, MSE = 0.3429
Saved: results\layer2\voxel_correlations_LH_IFG_layer2_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer2\correlation_map_LH_IFG_layer2.png


## LH_MFG

In [57]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [58]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=2, fedorenko_zone='LH_MFG', save_dir='results/layer2')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=2,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer2'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:22<00:00,  1.13s/it]


Model saved: results\layer2\subject_1_layer2_zoneLH_MFG_model.pth
Full model saved: results\layer2\subject_1_layer2_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:35<00:00,  1.79s/it]


Model saved: results\layer2\subject_2_layer2_zoneLH_MFG_model.pth
Full model saved: results\layer2\subject_2_layer2_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.05s/it]


Model saved: results\layer2\subject_5_layer2_zoneLH_MFG_model.pth
Full model saved: results\layer2\subject_5_layer2_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1040, mean MSE = 0.3118
  Subject 2: mean r = 0.0980, mean MSE = 0.2651
  Subject 3: mean r = 0.1084, mean MSE = 0.3204
  Subject 4: mean r = 0.0739, mean MSE = 0.3444
Mean across subjects: r = 0.0961, MSE = 0.3104
Saved: results\layer2\voxel_correlations_LH_MFG_layer2_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer2\correlation_map_LH_MFG_layer2.png


## LH_AntTemp

In [59]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [60]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=2, fedorenko_zone='LH_AntTemp', save_dir='results/layer2')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=2,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer2'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:47<00:00,  2.36s/it]


Model saved: results\layer2\subject_1_layer2_zoneLH_AntTemp_model.pth
Full model saved: results\layer2\subject_1_layer2_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]


Model saved: results\layer2\subject_2_layer2_zoneLH_AntTemp_model.pth
Full model saved: results\layer2\subject_2_layer2_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it]


Model saved: results\layer2\subject_5_layer2_zoneLH_AntTemp_model.pth
Full model saved: results\layer2\subject_5_layer2_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.1954, mean MSE = 0.2905
  Subject 2: mean r = 0.1959, mean MSE = 0.2898
  Subject 3: mean r = 0.2427, mean MSE = 0.3437
  Subject 4: mean r = 0.1896, mean MSE = 0.2962
Mean across subjects: r = 0.2059, MSE = 0.3050
Saved: results\layer2\voxel_correlations_LH_AntTemp_layer2_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer2\correlation_map_LH_AntTemp_layer2.png


## LH_PostTemp

In [61]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [62]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=2, fedorenko_zone='LH_PostTemp', save_dir='results/layer2')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=2,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer2'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:33<00:00,  1.67s/it]


Model saved: results\layer2\subject_1_layer2_zoneLH_PostTemp_model.pth
Full model saved: results\layer2\subject_1_layer2_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.65s/it]


Model saved: results\layer2\subject_2_layer2_zoneLH_PostTemp_model.pth
Full model saved: results\layer2\subject_2_layer2_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:48<00:00,  2.41s/it]


Model saved: results\layer2\subject_5_layer2_zoneLH_PostTemp_model.pth
Full model saved: results\layer2\subject_5_layer2_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1869, mean MSE = 0.3754
  Subject 2: mean r = 0.2030, mean MSE = 0.3474
  Subject 3: mean r = 0.2304, mean MSE = 0.3984
  Subject 4: mean r = 0.1998, mean MSE = 0.3610
Mean across subjects: r = 0.2050, MSE = 0.3706
Saved: results\layer2\voxel_correlations_LH_PostTemp_layer2_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer2\correlation_map_LH_PostTemp_layer2.png


# Layer 3

In [63]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_3')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:17,  3.51s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [64]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [65]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=3, fedorenko_zone='LH_IFGorb', save_dir='results/layer3')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=3,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer3'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:27<00:00,  1.39s/it]


Model saved: results\layer3\subject_1_layer3_zoneLH_IFGorb_model.pth
Full model saved: results\layer3\subject_1_layer3_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:29<00:00,  1.46s/it]


Model saved: results\layer3\subject_2_layer3_zoneLH_IFGorb_model.pth
Full model saved: results\layer3\subject_2_layer3_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.50s/it]


Model saved: results\layer3\subject_5_layer3_zoneLH_IFGorb_model.pth
Full model saved: results\layer3\subject_5_layer3_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0765, mean MSE = 0.3419
  Subject 2: mean r = 0.0830, mean MSE = 0.3054
  Subject 3: mean r = 0.0805, mean MSE = 0.3624
  Subject 4: mean r = 0.0513, mean MSE = 0.4007
Mean across subjects: r = 0.0728, MSE = 0.3526
Saved: results\layer3\voxel_correlations_LH_IFGorb_layer3_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer3\correlation_map_LH_IFGorb_layer3.png


## LH_IFG

In [66]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [67]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=3, fedorenko_zone='LH_IFG', save_dir='results/layer3')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=3,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer3'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.50s/it]


Model saved: results\layer3\subject_1_layer3_zoneLH_IFG_model.pth
Full model saved: results\layer3\subject_1_layer3_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it]


Model saved: results\layer3\subject_2_layer3_zoneLH_IFG_model.pth
Full model saved: results\layer3\subject_2_layer3_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.90s/it]


Model saved: results\layer3\subject_5_layer3_zoneLH_IFG_model.pth
Full model saved: results\layer3\subject_5_layer3_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1218, mean MSE = 0.3351
  Subject 2: mean r = 0.1372, mean MSE = 0.3018
  Subject 3: mean r = 0.1394, mean MSE = 0.3892
  Subject 4: mean r = 0.1063, mean MSE = 0.3445
Mean across subjects: r = 0.1262, MSE = 0.3427
Saved: results\layer3\voxel_correlations_LH_IFG_layer3_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer3\correlation_map_LH_IFG_layer3.png


## LH_MFG

In [68]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [69]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=3, fedorenko_zone='LH_MFG', save_dir='results/layer3')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=3,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer3'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.60s/it]


Model saved: results\layer3\subject_1_layer3_zoneLH_MFG_model.pth
Full model saved: results\layer3\subject_1_layer3_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.63s/it]


Model saved: results\layer3\subject_2_layer3_zoneLH_MFG_model.pth
Full model saved: results\layer3\subject_2_layer3_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.86s/it]


Model saved: results\layer3\subject_5_layer3_zoneLH_MFG_model.pth
Full model saved: results\layer3\subject_5_layer3_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1057, mean MSE = 0.3115
  Subject 2: mean r = 0.0988, mean MSE = 0.2651
  Subject 3: mean r = 0.1095, mean MSE = 0.3202
  Subject 4: mean r = 0.0761, mean MSE = 0.3442
Mean across subjects: r = 0.0975, MSE = 0.3103
Saved: results\layer3\voxel_correlations_LH_MFG_layer3_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer3\correlation_map_LH_MFG_layer3.png


## LH_AntTemp

In [70]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [71]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=3, fedorenko_zone='LH_AntTemp', save_dir='results/layer3')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=3,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer3'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.71s/it]


Model saved: results\layer3\subject_1_layer3_zoneLH_AntTemp_model.pth
Full model saved: results\layer3\subject_1_layer3_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:33<00:00,  1.68s/it]


Model saved: results\layer3\subject_2_layer3_zoneLH_AntTemp_model.pth
Full model saved: results\layer3\subject_2_layer3_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.15s/it]


Model saved: results\layer3\subject_5_layer3_zoneLH_AntTemp_model.pth
Full model saved: results\layer3\subject_5_layer3_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.1978, mean MSE = 0.2894
  Subject 2: mean r = 0.1998, mean MSE = 0.2890
  Subject 3: mean r = 0.2472, mean MSE = 0.3427
  Subject 4: mean r = 0.1920, mean MSE = 0.2953
Mean across subjects: r = 0.2092, MSE = 0.3041
Saved: results\layer3\voxel_correlations_LH_AntTemp_layer3_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer3\correlation_map_LH_AntTemp_layer3.png


## LH_PostTemp

In [72]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [73]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=3, fedorenko_zone='LH_PostTemp', save_dir='results/layer3')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=3,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer3'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]


Model saved: results\layer3\subject_1_layer3_zoneLH_PostTemp_model.pth
Full model saved: results\layer3\subject_1_layer3_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


Model saved: results\layer3\subject_2_layer3_zoneLH_PostTemp_model.pth
Full model saved: results\layer3\subject_2_layer3_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:33<00:00,  1.70s/it]


Model saved: results\layer3\subject_5_layer3_zoneLH_PostTemp_model.pth
Full model saved: results\layer3\subject_5_layer3_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1900, mean MSE = 0.3749
  Subject 2: mean r = 0.2055, mean MSE = 0.3471
  Subject 3: mean r = 0.2348, mean MSE = 0.3971
  Subject 4: mean r = 0.2039, mean MSE = 0.3604
Mean across subjects: r = 0.2085, MSE = 0.3699
Saved: results\layer3\voxel_correlations_LH_PostTemp_layer3_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer3\correlation_map_LH_PostTemp_layer3.png


# Layer 4

In [74]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_4')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:40,  8.05s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [75]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [76]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=4, fedorenko_zone='LH_IFGorb', save_dir='results/layer4')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=4,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer4'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.81s/it]


Model saved: results\layer4\subject_1_layer4_zoneLH_IFGorb_model.pth
Full model saved: results\layer4\subject_1_layer4_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.90s/it]


Model saved: results\layer4\subject_2_layer4_zoneLH_IFGorb_model.pth
Full model saved: results\layer4\subject_2_layer4_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]


Model saved: results\layer4\subject_5_layer4_zoneLH_IFGorb_model.pth
Full model saved: results\layer4\subject_5_layer4_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0745, mean MSE = 0.3420
  Subject 2: mean r = 0.0901, mean MSE = 0.3051
  Subject 3: mean r = 0.0789, mean MSE = 0.3626
  Subject 4: mean r = 0.0554, mean MSE = 0.4005
Mean across subjects: r = 0.0747, MSE = 0.3525
Saved: results\layer4\voxel_correlations_LH_IFGorb_layer4_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer4\correlation_map_LH_IFGorb_layer4.png


## LH_IFG

In [77]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [78]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=4, fedorenko_zone='LH_IFG', save_dir='results/layer4')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=4,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer4'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.55s/it]


Model saved: results\layer4\subject_1_layer4_zoneLH_IFG_model.pth
Full model saved: results\layer4\subject_1_layer4_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it]


Model saved: results\layer4\subject_2_layer4_zoneLH_IFG_model.pth
Full model saved: results\layer4\subject_2_layer4_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.57s/it]


Model saved: results\layer4\subject_5_layer4_zoneLH_IFG_model.pth
Full model saved: results\layer4\subject_5_layer4_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1275, mean MSE = 0.3345
  Subject 2: mean r = 0.1426, mean MSE = 0.3013
  Subject 3: mean r = 0.1437, mean MSE = 0.3887
  Subject 4: mean r = 0.1098, mean MSE = 0.3442
Mean across subjects: r = 0.1309, MSE = 0.3422
Saved: results\layer4\voxel_correlations_LH_IFG_layer4_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer4\correlation_map_LH_IFG_layer4.png


## LH_MFG

In [79]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [80]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=4, fedorenko_zone='LH_MFG', save_dir='results/layer4')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=4,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer4'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.57s/it]


Model saved: results\layer4\subject_1_layer4_zoneLH_MFG_model.pth
Full model saved: results\layer4\subject_1_layer4_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]


Model saved: results\layer4\subject_2_layer4_zoneLH_MFG_model.pth
Full model saved: results\layer4\subject_2_layer4_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.05s/it]


Model saved: results\layer4\subject_5_layer4_zoneLH_MFG_model.pth
Full model saved: results\layer4\subject_5_layer4_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1075, mean MSE = 0.3113
  Subject 2: mean r = 0.1017, mean MSE = 0.2649
  Subject 3: mean r = 0.1158, mean MSE = 0.3197
  Subject 4: mean r = 0.0800, mean MSE = 0.3440
Mean across subjects: r = 0.1012, MSE = 0.3100
Saved: results\layer4\voxel_correlations_LH_MFG_layer4_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer4\correlation_map_LH_MFG_layer4.png


## LH_AntTemp

In [81]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [82]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=4, fedorenko_zone='LH_AntTemp', save_dir='results/layer4')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=4,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer4'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.95s/it]


Model saved: results\layer4\subject_1_layer4_zoneLH_AntTemp_model.pth
Full model saved: results\layer4\subject_1_layer4_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.93s/it]


Model saved: results\layer4\subject_2_layer4_zoneLH_AntTemp_model.pth
Full model saved: results\layer4\subject_2_layer4_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.01s/it]


Model saved: results\layer4\subject_5_layer4_zoneLH_AntTemp_model.pth
Full model saved: results\layer4\subject_5_layer4_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.1987, mean MSE = 0.2901
  Subject 2: mean r = 0.2016, mean MSE = 0.2886
  Subject 3: mean r = 0.2486, mean MSE = 0.3427
  Subject 4: mean r = 0.1963, mean MSE = 0.2947
Mean across subjects: r = 0.2113, MSE = 0.3040
Saved: results\layer4\voxel_correlations_LH_AntTemp_layer4_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer4\correlation_map_LH_AntTemp_layer4.png


## LH_PostTemp

In [83]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [84]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=4, fedorenko_zone='LH_PostTemp', save_dir='results/layer4')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=4,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer4'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.05s/it]


Model saved: results\layer4\subject_1_layer4_zoneLH_PostTemp_model.pth
Full model saved: results\layer4\subject_1_layer4_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.19s/it]


Model saved: results\layer4\subject_2_layer4_zoneLH_PostTemp_model.pth
Full model saved: results\layer4\subject_2_layer4_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.17s/it]


Model saved: results\layer4\subject_5_layer4_zoneLH_PostTemp_model.pth
Full model saved: results\layer4\subject_5_layer4_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1943, mean MSE = 0.3743
  Subject 2: mean r = 0.2092, mean MSE = 0.3464
  Subject 3: mean r = 0.2371, mean MSE = 0.3967
  Subject 4: mean r = 0.2066, mean MSE = 0.3601
Mean across subjects: r = 0.2118, MSE = 0.3694
Saved: results\layer4\voxel_correlations_LH_PostTemp_layer4_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer4\correlation_map_LH_PostTemp_layer4.png


# Layer 5

In [85]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_5')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:30,  6.09s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [86]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [87]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=5, fedorenko_zone='LH_IFGorb', save_dir='results/layer5')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=5,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer5'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.71s/it]


Model saved: results\layer5\subject_1_layer5_zoneLH_IFGorb_model.pth
Full model saved: results\layer5\subject_1_layer5_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.81s/it]


Model saved: results\layer5\subject_2_layer5_zoneLH_IFGorb_model.pth
Full model saved: results\layer5\subject_2_layer5_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:45<00:00,  2.28s/it]


Model saved: results\layer5\subject_5_layer5_zoneLH_IFGorb_model.pth
Full model saved: results\layer5\subject_5_layer5_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0857, mean MSE = 0.3411
  Subject 2: mean r = 0.0934, mean MSE = 0.3048
  Subject 3: mean r = 0.0837, mean MSE = 0.3623
  Subject 4: mean r = 0.0591, mean MSE = 0.4001
Mean across subjects: r = 0.0805, MSE = 0.3521
Saved: results\layer5\voxel_correlations_LH_IFGorb_layer5_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer5\correlation_map_LH_IFGorb_layer5.png


## LH_IFG

In [88]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [89]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=5, fedorenko_zone='LH_IFG', save_dir='results/layer5')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=5,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer5'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:46<00:00,  2.34s/it]


Model saved: results\layer5\subject_1_layer5_zoneLH_IFG_model.pth
Full model saved: results\layer5\subject_1_layer5_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:49<00:00,  2.46s/it]


Model saved: results\layer5\subject_2_layer5_zoneLH_IFG_model.pth
Full model saved: results\layer5\subject_2_layer5_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]


Model saved: results\layer5\subject_5_layer5_zoneLH_IFG_model.pth
Full model saved: results\layer5\subject_5_layer5_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1326, mean MSE = 0.3339
  Subject 2: mean r = 0.1472, mean MSE = 0.3008
  Subject 3: mean r = 0.1518, mean MSE = 0.3877
  Subject 4: mean r = 0.1164, mean MSE = 0.3437
Mean across subjects: r = 0.1370, MSE = 0.3415
Saved: results\layer5\voxel_correlations_LH_IFG_layer5_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer5\correlation_map_LH_IFG_layer5.png


## LH_MFG

In [90]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [91]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=5, fedorenko_zone='LH_MFG', save_dir='results/layer5')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=5,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer5'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.52s/it]


Model saved: results\layer5\subject_1_layer5_zoneLH_MFG_model.pth
Full model saved: results\layer5\subject_1_layer5_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:33<00:00,  1.70s/it]


Model saved: results\layer5\subject_2_layer5_zoneLH_MFG_model.pth
Full model saved: results\layer5\subject_2_layer5_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.09s/it]


Model saved: results\layer5\subject_5_layer5_zoneLH_MFG_model.pth
Full model saved: results\layer5\subject_5_layer5_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1077, mean MSE = 0.3112
  Subject 2: mean r = 0.1059, mean MSE = 0.2647
  Subject 3: mean r = 0.1195, mean MSE = 0.3193
  Subject 4: mean r = 0.0854, mean MSE = 0.3436
Mean across subjects: r = 0.1046, MSE = 0.3097
Saved: results\layer5\voxel_correlations_LH_MFG_layer5_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer5\correlation_map_LH_MFG_layer5.png


## LH_AntTemp

In [92]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [93]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=5, fedorenko_zone='LH_AntTemp', save_dir='results/layer5')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=5,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer5'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]


Model saved: results\layer5\subject_1_layer5_zoneLH_AntTemp_model.pth
Full model saved: results\layer5\subject_1_layer5_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.93s/it]


Model saved: results\layer5\subject_2_layer5_zoneLH_AntTemp_model.pth
Full model saved: results\layer5\subject_2_layer5_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.18s/it]


Model saved: results\layer5\subject_5_layer5_zoneLH_AntTemp_model.pth
Full model saved: results\layer5\subject_5_layer5_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.2030, mean MSE = 0.2886
  Subject 2: mean r = 0.2052, mean MSE = 0.2881
  Subject 3: mean r = 0.2517, mean MSE = 0.3420
  Subject 4: mean r = 0.2024, mean MSE = 0.2938
Mean across subjects: r = 0.2156, MSE = 0.3031
Saved: results\layer5\voxel_correlations_LH_AntTemp_layer5_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer5\correlation_map_LH_AntTemp_layer5.png


## LH_PostTemp

In [94]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [95]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=5, fedorenko_zone='LH_PostTemp', save_dir='results/layer5')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=5,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer5'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.97s/it]


Model saved: results\layer5\subject_1_layer5_zoneLH_PostTemp_model.pth
Full model saved: results\layer5\subject_1_layer5_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:44<00:00,  2.23s/it]


Model saved: results\layer5\subject_2_layer5_zoneLH_PostTemp_model.pth
Full model saved: results\layer5\subject_2_layer5_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.86s/it]


Model saved: results\layer5\subject_5_layer5_zoneLH_PostTemp_model.pth
Full model saved: results\layer5\subject_5_layer5_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1972, mean MSE = 0.3739
  Subject 2: mean r = 0.2142, mean MSE = 0.3454
  Subject 3: mean r = 0.2420, mean MSE = 0.3958
  Subject 4: mean r = 0.2119, mean MSE = 0.3591
Mean across subjects: r = 0.2163, MSE = 0.3685
Saved: results\layer5\voxel_correlations_LH_PostTemp_layer5_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer5\correlation_map_LH_PostTemp_layer5.png


# Layer 6

In [96]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_6')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:37,  7.40s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [97]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [98]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=6, fedorenko_zone='LH_IFGorb', save_dir='results/layer6')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=6,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer6'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.65s/it]


Model saved: results\layer6\subject_1_layer6_zoneLH_IFGorb_model.pth
Full model saved: results\layer6\subject_1_layer6_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.94s/it]


Model saved: results\layer6\subject_2_layer6_zoneLH_IFGorb_model.pth
Full model saved: results\layer6\subject_2_layer6_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.97s/it]


Model saved: results\layer6\subject_5_layer6_zoneLH_IFGorb_model.pth
Full model saved: results\layer6\subject_5_layer6_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0845, mean MSE = 0.3412
  Subject 2: mean r = 0.0946, mean MSE = 0.3048
  Subject 3: mean r = 0.0850, mean MSE = 0.3620
  Subject 4: mean r = 0.0622, mean MSE = 0.4000
Mean across subjects: r = 0.0816, MSE = 0.3520
Saved: results\layer6\voxel_correlations_LH_IFGorb_layer6_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer6\correlation_map_LH_IFGorb_layer6.png


## LH_IFG

In [99]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [100]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=6, fedorenko_zone='LH_IFG', save_dir='results/layer6')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=6,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer6'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.17s/it]


Model saved: results\layer6\subject_1_layer6_zoneLH_IFG_model.pth
Full model saved: results\layer6\subject_1_layer6_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:48<00:00,  2.41s/it]


Model saved: results\layer6\subject_2_layer6_zoneLH_IFG_model.pth
Full model saved: results\layer6\subject_2_layer6_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]


Model saved: results\layer6\subject_5_layer6_zoneLH_IFG_model.pth
Full model saved: results\layer6\subject_5_layer6_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1333, mean MSE = 0.3339
  Subject 2: mean r = 0.1514, mean MSE = 0.3004
  Subject 3: mean r = 0.1509, mean MSE = 0.3877
  Subject 4: mean r = 0.1160, mean MSE = 0.3437
Mean across subjects: r = 0.1379, MSE = 0.3415
Saved: results\layer6\voxel_correlations_LH_IFG_layer6_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer6\correlation_map_LH_IFG_layer6.png


## LH_MFG

In [101]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [102]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=6, fedorenko_zone='LH_MFG', save_dir='results/layer6')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=6,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer6'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:35<00:00,  1.77s/it]


Model saved: results\layer6\subject_1_layer6_zoneLH_MFG_model.pth
Full model saved: results\layer6\subject_1_layer6_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.87s/it]


Model saved: results\layer6\subject_2_layer6_zoneLH_MFG_model.pth
Full model saved: results\layer6\subject_2_layer6_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.07s/it]


Model saved: results\layer6\subject_5_layer6_zoneLH_MFG_model.pth
Full model saved: results\layer6\subject_5_layer6_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1105, mean MSE = 0.3110
  Subject 2: mean r = 0.1115, mean MSE = 0.2644
  Subject 3: mean r = 0.1207, mean MSE = 0.3192
  Subject 4: mean r = 0.0848, mean MSE = 0.3436
Mean across subjects: r = 0.1069, MSE = 0.3095
Saved: results\layer6\voxel_correlations_LH_MFG_layer6_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer6\correlation_map_LH_MFG_layer6.png


## LH_AntTemp

In [103]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [104]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=6, fedorenko_zone='LH_AntTemp', save_dir='results/layer6')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=6,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer6'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.63s/it]


Model saved: results\layer6\subject_1_layer6_zoneLH_AntTemp_model.pth
Full model saved: results\layer6\subject_1_layer6_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]


Model saved: results\layer6\subject_2_layer6_zoneLH_AntTemp_model.pth
Full model saved: results\layer6\subject_2_layer6_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it]


Model saved: results\layer6\subject_5_layer6_zoneLH_AntTemp_model.pth
Full model saved: results\layer6\subject_5_layer6_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.2017, mean MSE = 0.2888
  Subject 2: mean r = 0.2074, mean MSE = 0.2875
  Subject 3: mean r = 0.2475, mean MSE = 0.3429
  Subject 4: mean r = 0.2011, mean MSE = 0.2938
Mean across subjects: r = 0.2144, MSE = 0.3032
Saved: results\layer6\voxel_correlations_LH_AntTemp_layer6_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer6\correlation_map_LH_AntTemp_layer6.png


## LH_PostTemp

In [105]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [106]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=6, fedorenko_zone='LH_PostTemp', save_dir='results/layer6')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=6,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer6'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:32<00:00,  1.60s/it]


Model saved: results\layer6\subject_1_layer6_zoneLH_PostTemp_model.pth
Full model saved: results\layer6\subject_1_layer6_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.60s/it]


Model saved: results\layer6\subject_2_layer6_zoneLH_PostTemp_model.pth
Full model saved: results\layer6\subject_2_layer6_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]


Model saved: results\layer6\subject_5_layer6_zoneLH_PostTemp_model.pth
Full model saved: results\layer6\subject_5_layer6_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1980, mean MSE = 0.3736
  Subject 2: mean r = 0.2146, mean MSE = 0.3456
  Subject 3: mean r = 0.2432, mean MSE = 0.3958
  Subject 4: mean r = 0.2157, mean MSE = 0.3585
Mean across subjects: r = 0.2179, MSE = 0.3684
Saved: results\layer6\voxel_correlations_LH_PostTemp_layer6_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer6\correlation_map_LH_PostTemp_layer6.png


# Layer 7

In [107]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_7')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:27,  5.47s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [108]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [109]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=7, fedorenko_zone='LH_IFGorb', save_dir='results/layer7')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=7,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer7'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.72s/it]


Model saved: results\layer7\subject_1_layer7_zoneLH_IFGorb_model.pth
Full model saved: results\layer7\subject_1_layer7_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.95s/it]


Model saved: results\layer7\subject_2_layer7_zoneLH_IFGorb_model.pth
Full model saved: results\layer7\subject_2_layer7_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.84s/it]


Model saved: results\layer7\subject_5_layer7_zoneLH_IFGorb_model.pth
Full model saved: results\layer7\subject_5_layer7_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0942, mean MSE = 0.3404
  Subject 2: mean r = 0.0997, mean MSE = 0.3043
  Subject 3: mean r = 0.0925, mean MSE = 0.3615
  Subject 4: mean r = 0.0651, mean MSE = 0.3999
Mean across subjects: r = 0.0879, MSE = 0.3515
Saved: results\layer7\voxel_correlations_LH_IFGorb_layer7_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer7\correlation_map_LH_IFGorb_layer7.png


## LH_IFG

In [110]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [111]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=7, fedorenko_zone='LH_IFG', save_dir='results/layer7')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=7,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer7'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:35<00:00,  1.79s/it]


Model saved: results\layer7\subject_1_layer7_zoneLH_IFG_model.pth
Full model saved: results\layer7\subject_1_layer7_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:35<00:00,  1.79s/it]


Model saved: results\layer7\subject_2_layer7_zoneLH_IFG_model.pth
Full model saved: results\layer7\subject_2_layer7_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:34<00:00,  1.72s/it]


Model saved: results\layer7\subject_5_layer7_zoneLH_IFG_model.pth
Full model saved: results\layer7\subject_5_layer7_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1374, mean MSE = 0.3335
  Subject 2: mean r = 0.1589, mean MSE = 0.2998
  Subject 3: mean r = 0.1592, mean MSE = 0.3867
  Subject 4: mean r = 0.1237, mean MSE = 0.3430
Mean across subjects: r = 0.1448, MSE = 0.3408
Saved: results\layer7\voxel_correlations_LH_IFG_layer7_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer7\correlation_map_LH_IFG_layer7.png


## LH_MFG

In [112]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [113]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=7, fedorenko_zone='LH_MFG', save_dir='results/layer7')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=7,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer7'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:42<00:00,  2.12s/it]


Model saved: results\layer7\subject_1_layer7_zoneLH_MFG_model.pth
Full model saved: results\layer7\subject_1_layer7_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [01:12<00:00,  3.64s/it]


Model saved: results\layer7\subject_2_layer7_zoneLH_MFG_model.pth
Full model saved: results\layer7\subject_2_layer7_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.01s/it]


Model saved: results\layer7\subject_5_layer7_zoneLH_MFG_model.pth
Full model saved: results\layer7\subject_5_layer7_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1143, mean MSE = 0.3107
  Subject 2: mean r = 0.1131, mean MSE = 0.2642
  Subject 3: mean r = 0.1231, mean MSE = 0.3188
  Subject 4: mean r = 0.0911, mean MSE = 0.3433
Mean across subjects: r = 0.1104, MSE = 0.3092
Saved: results\layer7\voxel_correlations_LH_MFG_layer7_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer7\correlation_map_LH_MFG_layer7.png


## LH_AntTemp

In [114]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [115]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=7, fedorenko_zone='LH_AntTemp', save_dir='results/layer7')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=7,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer7'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it]


Model saved: results\layer7\subject_1_layer7_zoneLH_AntTemp_model.pth
Full model saved: results\layer7\subject_1_layer7_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it]


Model saved: results\layer7\subject_2_layer7_zoneLH_AntTemp_model.pth
Full model saved: results\layer7\subject_2_layer7_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:28<00:00,  1.41s/it]


Model saved: results\layer7\subject_5_layer7_zoneLH_AntTemp_model.pth
Full model saved: results\layer7\subject_5_layer7_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.2053, mean MSE = 0.2885
  Subject 2: mean r = 0.2148, mean MSE = 0.2865
  Subject 3: mean r = 0.2560, mean MSE = 0.3411
  Subject 4: mean r = 0.2063, mean MSE = 0.2930
Mean across subjects: r = 0.2206, MSE = 0.3023
Saved: results\layer7\voxel_correlations_LH_AntTemp_layer7_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer7\correlation_map_LH_AntTemp_layer7.png


## LH_PostTemp

In [116]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [117]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=7, fedorenko_zone='LH_PostTemp', save_dir='results/layer7')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=7,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer7'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:29<00:00,  1.47s/it]


Model saved: results\layer7\subject_1_layer7_zoneLH_PostTemp_model.pth
Full model saved: results\layer7\subject_1_layer7_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:28<00:00,  1.44s/it]


Model saved: results\layer7\subject_2_layer7_zoneLH_PostTemp_model.pth
Full model saved: results\layer7\subject_2_layer7_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:28<00:00,  1.44s/it]


Model saved: results\layer7\subject_5_layer7_zoneLH_PostTemp_model.pth
Full model saved: results\layer7\subject_5_layer7_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.2034, mean MSE = 0.3729
  Subject 2: mean r = 0.2226, mean MSE = 0.3440
  Subject 3: mean r = 0.2489, mean MSE = 0.3945
  Subject 4: mean r = 0.2181, mean MSE = 0.3580
Mean across subjects: r = 0.2232, MSE = 0.3674
Saved: results\layer7\voxel_correlations_LH_PostTemp_layer7_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer7\correlation_map_LH_PostTemp_layer7.png


# Layer 8

In [118]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_8')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:24,  4.99s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [119]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [120]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=8, fedorenko_zone='LH_IFGorb', save_dir='results/layer8')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=8,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer8'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:27<00:00,  1.40s/it]


Model saved: results\layer8\subject_1_layer8_zoneLH_IFGorb_model.pth
Full model saved: results\layer8\subject_1_layer8_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]


Model saved: results\layer8\subject_2_layer8_zoneLH_IFGorb_model.pth
Full model saved: results\layer8\subject_2_layer8_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:35<00:00,  1.75s/it]


Model saved: results\layer8\subject_5_layer8_zoneLH_IFGorb_model.pth
Full model saved: results\layer8\subject_5_layer8_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0957, mean MSE = 0.3405
  Subject 2: mean r = 0.0991, mean MSE = 0.3044
  Subject 3: mean r = 0.0895, mean MSE = 0.3619
  Subject 4: mean r = 0.0605, mean MSE = 0.4002
Mean across subjects: r = 0.0862, MSE = 0.3517
Saved: results\layer8\voxel_correlations_LH_IFGorb_layer8_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer8\correlation_map_LH_IFGorb_layer8.png


## LH_IFG

In [121]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [122]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=8, fedorenko_zone='LH_IFG', save_dir='results/layer8')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=8,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer8'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.07s/it]


Model saved: results\layer8\subject_1_layer8_zoneLH_IFG_model.pth
Full model saved: results\layer8\subject_1_layer8_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.90s/it]


Model saved: results\layer8\subject_2_layer8_zoneLH_IFG_model.pth
Full model saved: results\layer8\subject_2_layer8_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.86s/it]


Model saved: results\layer8\subject_5_layer8_zoneLH_IFG_model.pth
Full model saved: results\layer8\subject_5_layer8_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1394, mean MSE = 0.3333
  Subject 2: mean r = 0.1575, mean MSE = 0.2998
  Subject 3: mean r = 0.1577, mean MSE = 0.3869
  Subject 4: mean r = 0.1252, mean MSE = 0.3429
Mean across subjects: r = 0.1449, MSE = 0.3407
Saved: results\layer8\voxel_correlations_LH_IFG_layer8_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer8\correlation_map_LH_IFG_layer8.png


## LH_MFG

In [123]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [124]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=8, fedorenko_zone='LH_MFG', save_dir='results/layer8')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=8,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer8'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.95s/it]


Model saved: results\layer8\subject_1_layer8_zoneLH_MFG_model.pth
Full model saved: results\layer8\subject_1_layer8_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.92s/it]


Model saved: results\layer8\subject_2_layer8_zoneLH_MFG_model.pth
Full model saved: results\layer8\subject_2_layer8_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.02s/it]


Model saved: results\layer8\subject_5_layer8_zoneLH_MFG_model.pth
Full model saved: results\layer8\subject_5_layer8_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1155, mean MSE = 0.3107
  Subject 2: mean r = 0.1125, mean MSE = 0.2643
  Subject 3: mean r = 0.1232, mean MSE = 0.3188
  Subject 4: mean r = 0.0889, mean MSE = 0.3435
Mean across subjects: r = 0.1100, MSE = 0.3093
Saved: results\layer8\voxel_correlations_LH_MFG_layer8_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer8\correlation_map_LH_MFG_layer8.png


## LH_AntTemp

In [125]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [126]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=8, fedorenko_zone='LH_AntTemp', save_dir='results/layer8')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=8,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer8'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.91s/it]


Model saved: results\layer8\subject_1_layer8_zoneLH_AntTemp_model.pth
Full model saved: results\layer8\subject_1_layer8_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.84s/it]


Model saved: results\layer8\subject_2_layer8_zoneLH_AntTemp_model.pth
Full model saved: results\layer8\subject_2_layer8_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.93s/it]


Model saved: results\layer8\subject_5_layer8_zoneLH_AntTemp_model.pth
Full model saved: results\layer8\subject_5_layer8_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.2041, mean MSE = 0.2890
  Subject 2: mean r = 0.2140, mean MSE = 0.2865
  Subject 3: mean r = 0.2542, mean MSE = 0.3418
  Subject 4: mean r = 0.2076, mean MSE = 0.2930
Mean across subjects: r = 0.2200, MSE = 0.3026
Saved: results\layer8\voxel_correlations_LH_AntTemp_layer8_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer8\correlation_map_LH_AntTemp_layer8.png


## LH_PostTemp

In [127]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [128]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=8, fedorenko_zone='LH_PostTemp', save_dir='results/layer8')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=8,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer8'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.07s/it]


Model saved: results\layer8\subject_1_layer8_zoneLH_PostTemp_model.pth
Full model saved: results\layer8\subject_1_layer8_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.04s/it]


Model saved: results\layer8\subject_2_layer8_zoneLH_PostTemp_model.pth
Full model saved: results\layer8\subject_2_layer8_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.95s/it]


Model saved: results\layer8\subject_5_layer8_zoneLH_PostTemp_model.pth
Full model saved: results\layer8\subject_5_layer8_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.2018, mean MSE = 0.3732
  Subject 2: mean r = 0.2178, mean MSE = 0.3451
  Subject 3: mean r = 0.2476, mean MSE = 0.3948
  Subject 4: mean r = 0.2188, mean MSE = 0.3579
Mean across subjects: r = 0.2215, MSE = 0.3677
Saved: results\layer8\voxel_correlations_LH_PostTemp_layer8_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer8\correlation_map_LH_PostTemp_layer8.png


# Layer 9

In [129]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_9')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:28,  5.73s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [130]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [131]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=9, fedorenko_zone='LH_IFGorb', save_dir='results/layer9')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=9,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer9'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.07s/it]


Model saved: results\layer9\subject_1_layer9_zoneLH_IFGorb_model.pth
Full model saved: results\layer9\subject_1_layer9_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:42<00:00,  2.15s/it]


Model saved: results\layer9\subject_2_layer9_zoneLH_IFGorb_model.pth
Full model saved: results\layer9\subject_2_layer9_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:42<00:00,  2.11s/it]


Model saved: results\layer9\subject_5_layer9_zoneLH_IFGorb_model.pth
Full model saved: results\layer9\subject_5_layer9_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0893, mean MSE = 0.3408
  Subject 2: mean r = 0.1022, mean MSE = 0.3042
  Subject 3: mean r = 0.0869, mean MSE = 0.3619
  Subject 4: mean r = 0.0659, mean MSE = 0.3997
Mean across subjects: r = 0.0861, MSE = 0.3516
Saved: results\layer9\voxel_correlations_LH_IFGorb_layer9_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer9\correlation_map_LH_IFGorb_layer9.png


## LH_IFG

In [132]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [133]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=9, fedorenko_zone='LH_IFG', save_dir='results/layer9')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=9,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer9'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:45<00:00,  2.29s/it]


Model saved: results\layer9\subject_1_layer9_zoneLH_IFG_model.pth
Full model saved: results\layer9\subject_1_layer9_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.19s/it]


Model saved: results\layer9\subject_2_layer9_zoneLH_IFG_model.pth
Full model saved: results\layer9\subject_2_layer9_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.16s/it]


Model saved: results\layer9\subject_5_layer9_zoneLH_IFG_model.pth
Full model saved: results\layer9\subject_5_layer9_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1399, mean MSE = 0.3333
  Subject 2: mean r = 0.1577, mean MSE = 0.2998
  Subject 3: mean r = 0.1543, mean MSE = 0.3873
  Subject 4: mean r = 0.1250, mean MSE = 0.3429
Mean across subjects: r = 0.1442, MSE = 0.3408
Saved: results\layer9\voxel_correlations_LH_IFG_layer9_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer9\correlation_map_LH_IFG_layer9.png


## LH_MFG

In [134]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [135]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=9, fedorenko_zone='LH_MFG', save_dir='results/layer9')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=9,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer9'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.06s/it]


Model saved: results\layer9\subject_1_layer9_zoneLH_MFG_model.pth
Full model saved: results\layer9\subject_1_layer9_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:42<00:00,  2.14s/it]


Model saved: results\layer9\subject_2_layer9_zoneLH_MFG_model.pth
Full model saved: results\layer9\subject_2_layer9_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  2.00s/it]


Model saved: results\layer9\subject_5_layer9_zoneLH_MFG_model.pth
Full model saved: results\layer9\subject_5_layer9_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1141, mean MSE = 0.3107
  Subject 2: mean r = 0.1107, mean MSE = 0.2644
  Subject 3: mean r = 0.1249, mean MSE = 0.3187
  Subject 4: mean r = 0.0905, mean MSE = 0.3433
Mean across subjects: r = 0.1101, MSE = 0.3093
Saved: results\layer9\voxel_correlations_LH_MFG_layer9_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer9\correlation_map_LH_MFG_layer9.png


## LH_AntTemp

In [136]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [137]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=9, fedorenko_zone='LH_AntTemp', save_dir='results/layer9')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=9,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer9'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.05s/it]


Model saved: results\layer9\subject_1_layer9_zoneLH_AntTemp_model.pth
Full model saved: results\layer9\subject_1_layer9_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.05s/it]


Model saved: results\layer9\subject_2_layer9_zoneLH_AntTemp_model.pth
Full model saved: results\layer9\subject_2_layer9_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.05s/it]


Model saved: results\layer9\subject_5_layer9_zoneLH_AntTemp_model.pth
Full model saved: results\layer9\subject_5_layer9_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.2022, mean MSE = 0.2891
  Subject 2: mean r = 0.2121, mean MSE = 0.2871
  Subject 3: mean r = 0.2517, mean MSE = 0.3423
  Subject 4: mean r = 0.2075, mean MSE = 0.2929
Mean across subjects: r = 0.2184, MSE = 0.3029
Saved: results\layer9\voxel_correlations_LH_AntTemp_layer9_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer9\correlation_map_LH_AntTemp_layer9.png


## LH_PostTemp

In [138]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [139]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=9, fedorenko_zone='LH_PostTemp', save_dir='results/layer9')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=9,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer9'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.19s/it]


Model saved: results\layer9\subject_1_layer9_zoneLH_PostTemp_model.pth
Full model saved: results\layer9\subject_1_layer9_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:42<00:00,  2.15s/it]


Model saved: results\layer9\subject_2_layer9_zoneLH_PostTemp_model.pth
Full model saved: results\layer9\subject_2_layer9_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:42<00:00,  2.14s/it]


Model saved: results\layer9\subject_5_layer9_zoneLH_PostTemp_model.pth
Full model saved: results\layer9\subject_5_layer9_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.2023, mean MSE = 0.3732
  Subject 2: mean r = 0.2190, mean MSE = 0.3449
  Subject 3: mean r = 0.2444, mean MSE = 0.3958
  Subject 4: mean r = 0.2156, mean MSE = 0.3586
Mean across subjects: r = 0.2203, MSE = 0.3681
Saved: results\layer9\voxel_correlations_LH_PostTemp_layer9_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer9\correlation_map_LH_PostTemp_layer9.png


# Layer 10

In [140]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_10')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:29,  5.82s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [141]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [142]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=10, fedorenko_zone='LH_IFGorb', save_dir='results/layer10')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=10,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer10'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.02s/it]


Model saved: results\layer10\subject_1_layer10_zoneLH_IFGorb_model.pth
Full model saved: results\layer10\subject_1_layer10_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.04s/it]


Model saved: results\layer10\subject_2_layer10_zoneLH_IFGorb_model.pth
Full model saved: results\layer10\subject_2_layer10_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.08s/it]


Model saved: results\layer10\subject_5_layer10_zoneLH_IFGorb_model.pth
Full model saved: results\layer10\subject_5_layer10_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0893, mean MSE = 0.3409
  Subject 2: mean r = 0.0932, mean MSE = 0.3047
  Subject 3: mean r = 0.0867, mean MSE = 0.3617
  Subject 4: mean r = 0.0641, mean MSE = 0.3999
Mean across subjects: r = 0.0834, MSE = 0.3518
Saved: results\layer10\voxel_correlations_LH_IFGorb_layer10_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer10\correlation_map_LH_IFGorb_layer10.png


## LH_IFG

In [143]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [144]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=10, fedorenko_zone='LH_IFG', save_dir='results/layer10')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=10,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer10'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.16s/it]


Model saved: results\layer10\subject_1_layer10_zoneLH_IFG_model.pth
Full model saved: results\layer10\subject_1_layer10_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.15s/it]


Model saved: results\layer10\subject_2_layer10_zoneLH_IFG_model.pth
Full model saved: results\layer10\subject_2_layer10_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.19s/it]


Model saved: results\layer10\subject_5_layer10_zoneLH_IFG_model.pth
Full model saved: results\layer10\subject_5_layer10_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1376, mean MSE = 0.3336
  Subject 2: mean r = 0.1576, mean MSE = 0.2999
  Subject 3: mean r = 0.1562, mean MSE = 0.3871
  Subject 4: mean r = 0.1237, mean MSE = 0.3430
Mean across subjects: r = 0.1438, MSE = 0.3409
Saved: results\layer10\voxel_correlations_LH_IFG_layer10_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer10\correlation_map_LH_IFG_layer10.png


## LH_MFG

In [145]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [146]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=10, fedorenko_zone='LH_MFG', save_dir='results/layer10')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=10,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer10'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.93s/it]


Model saved: results\layer10\subject_1_layer10_zoneLH_MFG_model.pth
Full model saved: results\layer10\subject_1_layer10_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.03s/it]


Model saved: results\layer10\subject_2_layer10_zoneLH_MFG_model.pth
Full model saved: results\layer10\subject_2_layer10_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.98s/it]


Model saved: results\layer10\subject_5_layer10_zoneLH_MFG_model.pth
Full model saved: results\layer10\subject_5_layer10_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1119, mean MSE = 0.3112
  Subject 2: mean r = 0.1107, mean MSE = 0.2644
  Subject 3: mean r = 0.1226, mean MSE = 0.3190
  Subject 4: mean r = 0.0916, mean MSE = 0.3433
Mean across subjects: r = 0.1092, MSE = 0.3095
Saved: results\layer10\voxel_correlations_LH_MFG_layer10_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer10\correlation_map_LH_MFG_layer10.png


## LH_AntTemp

In [147]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [148]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=10, fedorenko_zone='LH_AntTemp', save_dir='results/layer10')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=10,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer10'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.07s/it]


Model saved: results\layer10\subject_1_layer10_zoneLH_AntTemp_model.pth
Full model saved: results\layer10\subject_1_layer10_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.03s/it]


Model saved: results\layer10\subject_2_layer10_zoneLH_AntTemp_model.pth
Full model saved: results\layer10\subject_2_layer10_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.00s/it]


Model saved: results\layer10\subject_5_layer10_zoneLH_AntTemp_model.pth
Full model saved: results\layer10\subject_5_layer10_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.1985, mean MSE = 0.2904
  Subject 2: mean r = 0.2100, mean MSE = 0.2874
  Subject 3: mean r = 0.2489, mean MSE = 0.3428
  Subject 4: mean r = 0.2042, mean MSE = 0.2937
Mean across subjects: r = 0.2154, MSE = 0.3036
Saved: results\layer10\voxel_correlations_LH_AntTemp_layer10_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer10\correlation_map_LH_AntTemp_layer10.png


## LH_PostTemp

In [149]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [150]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=10, fedorenko_zone='LH_PostTemp', save_dir='results/layer10')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=10,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer10'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.99s/it]


Model saved: results\layer10\subject_1_layer10_zoneLH_PostTemp_model.pth
Full model saved: results\layer10\subject_1_layer10_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.09s/it]


Model saved: results\layer10\subject_2_layer10_zoneLH_PostTemp_model.pth
Full model saved: results\layer10\subject_2_layer10_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [01:40<00:00,  5.01s/it]


Model saved: results\layer10\subject_5_layer10_zoneLH_PostTemp_model.pth
Full model saved: results\layer10\subject_5_layer10_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1979, mean MSE = 0.3739
  Subject 2: mean r = 0.2172, mean MSE = 0.3451
  Subject 3: mean r = 0.2426, mean MSE = 0.3963
  Subject 4: mean r = 0.2144, mean MSE = 0.3587
Mean across subjects: r = 0.2180, MSE = 0.3685
Saved: results\layer10\voxel_correlations_LH_PostTemp_layer10_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer10\correlation_map_LH_PostTemp_layer10.png


# Layer 11

In [151]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_11')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:45,  9.13s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [152]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [153]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=11, fedorenko_zone='LH_IFGorb', save_dir='results/layer11')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=11,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer11'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [01:51<00:00,  5.56s/it]


Model saved: results\layer11\subject_1_layer11_zoneLH_IFGorb_model.pth
Full model saved: results\layer11\subject_1_layer11_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [01:52<00:00,  5.65s/it]


Model saved: results\layer11\subject_2_layer11_zoneLH_IFGorb_model.pth
Full model saved: results\layer11\subject_2_layer11_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.07s/it]


Model saved: results\layer11\subject_5_layer11_zoneLH_IFGorb_model.pth
Full model saved: results\layer11\subject_5_layer11_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0861, mean MSE = 0.3412
  Subject 2: mean r = 0.0909, mean MSE = 0.3049
  Subject 3: mean r = 0.0891, mean MSE = 0.3617
  Subject 4: mean r = 0.0604, mean MSE = 0.4001
Mean across subjects: r = 0.0816, MSE = 0.3520
Saved: results\layer11\voxel_correlations_LH_IFGorb_layer11_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer11\correlation_map_LH_IFGorb_layer11.png


## LH_IFG

In [154]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [155]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=11, fedorenko_zone='LH_IFG', save_dir='results/layer11')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=11,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer11'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.99s/it]


Model saved: results\layer11\subject_1_layer11_zoneLH_IFG_model.pth
Full model saved: results\layer11\subject_1_layer11_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.20s/it]


Model saved: results\layer11\subject_2_layer11_zoneLH_IFG_model.pth
Full model saved: results\layer11\subject_2_layer11_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:47<00:00,  2.35s/it]


Model saved: results\layer11\subject_5_layer11_zoneLH_IFG_model.pth
Full model saved: results\layer11\subject_5_layer11_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1367, mean MSE = 0.3338
  Subject 2: mean r = 0.1537, mean MSE = 0.3003
  Subject 3: mean r = 0.1529, mean MSE = 0.3876
  Subject 4: mean r = 0.1207, mean MSE = 0.3433
Mean across subjects: r = 0.1410, MSE = 0.3412
Saved: results\layer11\voxel_correlations_LH_IFG_layer11_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer11\correlation_map_LH_IFG_layer11.png


## LH_MFG

In [156]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [157]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=11, fedorenko_zone='LH_MFG', save_dir='results/layer11')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=11,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer11'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.01s/it]


Model saved: results\layer11\subject_1_layer11_zoneLH_MFG_model.pth
Full model saved: results\layer11\subject_1_layer11_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [01:50<00:00,  5.52s/it]


Model saved: results\layer11\subject_2_layer11_zoneLH_MFG_model.pth
Full model saved: results\layer11\subject_2_layer11_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.81s/it]


Model saved: results\layer11\subject_5_layer11_zoneLH_MFG_model.pth
Full model saved: results\layer11\subject_5_layer11_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1093, mean MSE = 0.3113
  Subject 2: mean r = 0.1123, mean MSE = 0.2644
  Subject 3: mean r = 0.1227, mean MSE = 0.3190
  Subject 4: mean r = 0.0891, mean MSE = 0.3436
Mean across subjects: r = 0.1084, MSE = 0.3096
Saved: results\layer11\voxel_correlations_LH_MFG_layer11_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer11\correlation_map_LH_MFG_layer11.png


## LH_AntTemp

In [158]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [159]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=11, fedorenko_zone='LH_AntTemp', save_dir='results/layer11')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=11,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer11'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [01:11<00:00,  3.59s/it]


Model saved: results\layer11\subject_1_layer11_zoneLH_AntTemp_model.pth
Full model saved: results\layer11\subject_1_layer11_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:47<00:00,  2.38s/it]


Model saved: results\layer11\subject_2_layer11_zoneLH_AntTemp_model.pth
Full model saved: results\layer11\subject_2_layer11_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.86s/it]


Model saved: results\layer11\subject_5_layer11_zoneLH_AntTemp_model.pth
Full model saved: results\layer11\subject_5_layer11_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.1943, mean MSE = 0.2909
  Subject 2: mean r = 0.2000, mean MSE = 0.2894
  Subject 3: mean r = 0.2416, mean MSE = 0.3450
  Subject 4: mean r = 0.1982, mean MSE = 0.2943
Mean across subjects: r = 0.2085, MSE = 0.3049
Saved: results\layer11\voxel_correlations_LH_AntTemp_layer11_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer11\correlation_map_LH_AntTemp_layer11.png


## LH_PostTemp

In [160]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [161]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=11, fedorenko_zone='LH_PostTemp', save_dir='results/layer11')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=11,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer11'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.93s/it]


Model saved: results\layer11\subject_1_layer11_zoneLH_PostTemp_model.pth
Full model saved: results\layer11\subject_1_layer11_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.10s/it]


Model saved: results\layer11\subject_2_layer11_zoneLH_PostTemp_model.pth
Full model saved: results\layer11\subject_2_layer11_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:42<00:00,  2.13s/it]


Model saved: results\layer11\subject_5_layer11_zoneLH_PostTemp_model.pth
Full model saved: results\layer11\subject_5_layer11_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1951, mean MSE = 0.3746
  Subject 2: mean r = 0.2117, mean MSE = 0.3462
  Subject 3: mean r = 0.2376, mean MSE = 0.3973
  Subject 4: mean r = 0.2085, mean MSE = 0.3598
Mean across subjects: r = 0.2132, MSE = 0.3695
Saved: results\layer11\voxel_correlations_LH_PostTemp_layer11_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer11\correlation_map_LH_PostTemp_layer11.png


# Layer 12

In [162]:
features_train, features_test = load_stimulus_features(method='gpt2', layer='language_layer_12')

x_train_1, y_train_1, scaler_1 = align_features_and_fmri_samples(features_train, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay, movies_train)
x_train_2, y_train_2, scaler_2 = align_features_and_fmri_samples(features_train, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_3, y_train_3, scaler_3 = align_features_and_fmri_samples(features_train, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)
x_train_5, y_train_5, scaler_5 = align_features_and_fmri_samples(features_train, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_train)

x_test_1, y_test_1 = align_features_and_fmri_samples(features_test, fmri_1, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_1)
x_test_2, y_test_2 = align_features_and_fmri_samples(features_test, fmri_2, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_2)
x_test_3, y_test_3 = align_features_and_fmri_samples(features_test, fmri_3, excluded_samples_start, excluded_samples_end, hrf_delay, movies_test, scaler_3)
x_test_5, y_test_5 = align_features_and_fmri_samples(features_test, fmri_5, excluded_samples_start, excluded_samples_end, hrf_delay,  movies_test, scaler_5)

5it [00:32,  6.55s/it]


(114294, 1536)
(23387, 1536)
8070144
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (111343, 1536)
After NaN removal: (83980, 1536)
Removed 27363 NaN samples (24.6%)
Before NaN removal: (110900, 1536)
After NaN removal: (83652, 1536)
Removed 27248 NaN samples (24.6%)
Before NaN removal: (109998, 1536)
After NaN removal: (82994, 1536)
Removed 27004 NaN samples (24.5%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)
Before NaN removal: (22348, 1536)
After NaN removal: (17506, 1536)
Removed 4842 NaN samples (21.7%)
Before NaN removal: (22774, 1536)
After NaN removal: (17866, 1536)
Removed 4908 NaN samples (21.6%)


## LH_IFGorb

In [163]:
x_train_list = [x_train_1, x_train_2, x_train_3, x_train_5]
y_train_list = [y_train_1['LH_IFGorb'], y_train_2['LH_IFGorb'], y_train_3['LH_IFGorb'], y_train_5['LH_IFGorb']]
x_test_list = [x_test_1, x_test_2, x_test_3, x_test_5]
y_test_list = [y_test_1['LH_IFGorb'], y_test_2['LH_IFGorb'], y_test_3['LH_IFGorb'], y_test_5['LH_IFGorb']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFGorb',
    storage='sqlite:///optuna_mlp.db'
)

In [164]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=12, fedorenko_zone='LH_IFGorb', save_dir='results/layer12')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=12,
    fedorenko_zone='LH_IFGorb',
    output_dir='./results/layer12'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:43<00:00,  2.18s/it]


Model saved: results\layer12\subject_1_layer12_zoneLH_IFGorb_model.pth
Full model saved: results\layer12\subject_1_layer12_zoneLH_IFGorb_model_full.pkl
Subject 1 - Predictions shape: (17866, 6)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.82s/it]


Model saved: results\layer12\subject_2_layer12_zoneLH_IFGorb_model.pth
Full model saved: results\layer12\subject_2_layer12_zoneLH_IFGorb_model_full.pkl
Subject 2 - Predictions shape: (17866, 6)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:37<00:00,  1.89s/it]


Model saved: results\layer12\subject_5_layer12_zoneLH_IFGorb_model.pth
Full model saved: results\layer12\subject_5_layer12_zoneLH_IFGorb_model_full.pkl
Subject 5 - Predictions shape: (17866, 6)
Processing 4 subjects with 6 voxels each...
  Subject 1: mean r = 0.0825, mean MSE = 0.3418
  Subject 2: mean r = 0.0861, mean MSE = 0.3052
  Subject 3: mean r = 0.0803, mean MSE = 0.3623
  Subject 4: mean r = 0.0526, mean MSE = 0.4008
Mean across subjects: r = 0.0754, MSE = 0.3525
Saved: results\layer12\voxel_correlations_LH_IFGorb_layer12_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer12\correlation_map_LH_IFGorb_layer12.png


## LH_IFG

In [165]:
y_train_list = [y_train_1['LH_IFG'], y_train_2['LH_IFG'], y_train_3['LH_IFG'], y_train_5['LH_IFG']]
y_test_list = [y_test_1['LH_IFG'], y_test_2['LH_IFG'], y_test_3['LH_IFG'], y_test_5['LH_IFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_IFG',
    storage='sqlite:///optuna_mlp.db'
)

In [166]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=12, fedorenko_zone='LH_IFG', save_dir='results/layer12')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=12,
    fedorenko_zone='LH_IFG',
    output_dir='./results/layer12'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:44<00:00,  2.21s/it]


Model saved: results\layer12\subject_1_layer12_zoneLH_IFG_model.pth
Full model saved: results\layer12\subject_1_layer12_zoneLH_IFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 11)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.01s/it]


Model saved: results\layer12\subject_2_layer12_zoneLH_IFG_model.pth
Full model saved: results\layer12\subject_2_layer12_zoneLH_IFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 11)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [02:15<00:00,  6.75s/it]


Model saved: results\layer12\subject_5_layer12_zoneLH_IFG_model.pth
Full model saved: results\layer12\subject_5_layer12_zoneLH_IFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 11)
Processing 4 subjects with 11 voxels each...
  Subject 1: mean r = 0.1292, mean MSE = 0.3345
  Subject 2: mean r = 0.1434, mean MSE = 0.3013
  Subject 3: mean r = 0.1429, mean MSE = 0.3889
  Subject 4: mean r = 0.1156, mean MSE = 0.3438
Mean across subjects: r = 0.1328, MSE = 0.3421
Saved: results\layer12\voxel_correlations_LH_IFG_layer12_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer12\correlation_map_LH_IFG_layer12.png


## LH_MFG

In [167]:
y_train_list = [y_train_1['LH_MFG'], y_train_2['LH_MFG'], y_train_3['LH_MFG'], y_train_5['LH_MFG']]
y_test_list = [y_test_1['LH_MFG'], y_test_2['LH_MFG'], y_test_3['LH_MFG'], y_test_5['LH_MFG']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_MFG',
    storage='sqlite:///optuna_mlp.db'
)

In [168]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=12, fedorenko_zone='LH_MFG', save_dir='results/layer12')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=12,
    fedorenko_zone='LH_MFG',
    output_dir='./results/layer12'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [01:22<00:00,  4.12s/it]


Model saved: results\layer12\subject_1_layer12_zoneLH_MFG_model.pth
Full model saved: results\layer12\subject_1_layer12_zoneLH_MFG_model_full.pkl
Subject 1 - Predictions shape: (17866, 7)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.94s/it]


Model saved: results\layer12\subject_2_layer12_zoneLH_MFG_model.pth
Full model saved: results\layer12\subject_2_layer12_zoneLH_MFG_model_full.pkl
Subject 2 - Predictions shape: (17866, 7)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:38<00:00,  1.91s/it]


Model saved: results\layer12\subject_5_layer12_zoneLH_MFG_model.pth
Full model saved: results\layer12\subject_5_layer12_zoneLH_MFG_model_full.pkl
Subject 5 - Predictions shape: (17866, 7)
Processing 4 subjects with 7 voxels each...
  Subject 1: mean r = 0.1053, mean MSE = 0.3115
  Subject 2: mean r = 0.1024, mean MSE = 0.2649
  Subject 3: mean r = 0.1156, mean MSE = 0.3196
  Subject 4: mean r = 0.0817, mean MSE = 0.3440
Mean across subjects: r = 0.1013, MSE = 0.3100
Saved: results\layer12\voxel_correlations_LH_MFG_layer12_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer12\correlation_map_LH_MFG_layer12.png


## LH_AntTemp

In [169]:
y_train_list = [y_train_1['LH_AntTemp'], y_train_2['LH_AntTemp'], y_train_3['LH_AntTemp'], y_train_5['LH_AntTemp']]
y_test_list = [y_test_1['LH_AntTemp'], y_test_2['LH_AntTemp'], y_test_3['LH_AntTemp'], y_test_5['LH_AntTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_AntTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [170]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=12, fedorenko_zone='LH_AntTemp', save_dir='results/layer12')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=12,
    fedorenko_zone='LH_AntTemp',
    output_dir='./results/layer12'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:39<00:00,  1.96s/it]


Model saved: results\layer12\subject_1_layer12_zoneLH_AntTemp_model.pth
Full model saved: results\layer12\subject_1_layer12_zoneLH_AntTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 17)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.85s/it]


Model saved: results\layer12\subject_2_layer12_zoneLH_AntTemp_model.pth
Full model saved: results\layer12\subject_2_layer12_zoneLH_AntTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 17)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]


Model saved: results\layer12\subject_5_layer12_zoneLH_AntTemp_model.pth
Full model saved: results\layer12\subject_5_layer12_zoneLH_AntTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 17)
Processing 4 subjects with 17 voxels each...
  Subject 1: mean r = 0.1835, mean MSE = 0.2931
  Subject 2: mean r = 0.1932, mean MSE = 0.2906
  Subject 3: mean r = 0.2321, mean MSE = 0.3468
  Subject 4: mean r = 0.1894, mean MSE = 0.2963
Mean across subjects: r = 0.1995, MSE = 0.3067
Saved: results\layer12\voxel_correlations_LH_AntTemp_layer12_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer12\correlation_map_LH_AntTemp_layer12.png


## LH_PostTemp

In [171]:
y_train_list = [y_train_1['LH_PostTemp'], y_train_2['LH_PostTemp'], y_train_3['LH_PostTemp'], y_train_5['LH_PostTemp']]
y_test_list = [y_test_1['LH_PostTemp'], y_test_2['LH_PostTemp'], y_test_3['LH_PostTemp'], y_test_5['LH_PostTemp']]

study = optuna.load_study(
    study_name='fmri_optimization_l1_PostTemp',
    storage='sqlite:///optuna_mlp.db'
)

In [172]:
predictions_list = train_and_save_best_models(
    x_train_list, y_train_list, x_test_list, y_test_list,
    study, layer=12, fedorenko_zone='LH_PostTemp', save_dir='results/layer12')

results = plot_correlations(
    predictions_list=predictions_list,
    true_fmri_list=y_test_list,
    layer=12,
    fedorenko_zone='LH_PostTemp',
    output_dir='./results/layer12'
)

Training Subject 1


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.06s/it]


Model saved: results\layer12\subject_1_layer12_zoneLH_PostTemp_model.pth
Full model saved: results\layer12\subject_1_layer12_zoneLH_PostTemp_model_full.pkl
Subject 1 - Predictions shape: (17866, 40)
Training Subject 2


Training Epochs: 100%|██████████| 20/20 [00:41<00:00,  2.06s/it]


Model saved: results\layer12\subject_2_layer12_zoneLH_PostTemp_model.pth
Full model saved: results\layer12\subject_2_layer12_zoneLH_PostTemp_model_full.pkl
Subject 2 - Predictions shape: (17866, 40)
Training Subject 3


...

Training Epochs: 100%|██████████| 20/20 [00:40<00:00,  2.03s/it]


Model saved: results\layer12\subject_5_layer12_zoneLH_PostTemp_model.pth
Full model saved: results\layer12\subject_5_layer12_zoneLH_PostTemp_model_full.pkl
Subject 5 - Predictions shape: (17866, 40)
Processing 4 subjects with 40 voxels each...
  Subject 1: mean r = 0.1867, mean MSE = 0.3762
  Subject 2: mean r = 0.2036, mean MSE = 0.3475
  Subject 3: mean r = 0.2282, mean MSE = 0.3999
  Subject 4: mean r = 0.2013, mean MSE = 0.3611
Mean across subjects: r = 0.2050, MSE = 0.3712
Saved: results\layer12\voxel_correlations_LH_PostTemp_layer12_all_subjects.csv


[fetch_atlas_schaefer_2018] Dataset found in C:\Users\andre\nilearn_data\schaefer_2018

c:\Users\andre\OneDrive\Desktop\NLP\train_utils.py:334: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  display.add_overlay(


Saved: results\layer12\correlation_map_LH_PostTemp_layer12.png
